# QA of DSL Release v1.16
The purpose of this notebook is to review and test the new features in 1.16


## Prerequisites

> This notebook assumes you have installed the `dimcli` library and are familiar with the [Getting Started](https://github.com/digital-science/dimensions-api-examples/blob/master/stable/1.%20Getting%20Started.ipynb) tutorial. 

In [93]:
import dimcli
dimcli.login(instance="test")

DimCli v0.4.3 - Succesfully connected to <https://integration.ds-metrics.com>


---

## DSL-113

https://uberresearch.atlassian.net/browse/DSL-113

author_affiliations deprecated as a filter

In [7]:
%%dsl_query 
search publications where author_affiliations = "grid.10825.3e" 
return publications

Semantic Error
Semantic errors found:
	Field 'author_affiliations' can not be used in filters. It can only be returned in the result.


In [12]:
%%dsl_query 
search publications where research_orgs.id = "grid.10825.3e" 
return publications[author_affiliations+title] limit 10

Tot Publications:  29220
Returned Publications: 10


<dimcli.Result object #4529140624: [('publications', 10), ('_stats', 3)]>

In [10]:
infos = %dsl_query describe source publications
print(infos['fields']['author_affiliations'])

{'type': 'json', 'description': "List of JSON lists of researchers' first and last names and affiliations", 'long_description': None, 'is_entity': False, 'is_filter': False, 'is_facet': False}


### VERDICT
Looks good

---

## DSL-53

https://uberresearch.atlassian.net/browse/DSL-53

In [21]:
res = %dsl_query search policy_documents return policy_documents[date_inserted] sort by year
res['policy_documents'][0]

Tot Policy_documents:  422079
Returned Policy_documents: 20


{'date_inserted': '2017-01-17'}

In [88]:
%%dsl_query
search policy_documents where date_inserted="2017-01-17"

Tot Policy_documents:  0
Returned Policy_documents: 0


<dimcli.Result object #4531134304: [('policy_documents', 0), ('_stats', 1)]>

### ISSUES
* the field is available but it cannot be used to filter? the same seems to apply to publications.. so maybe I'd doing something wrong

In [92]:
%dsl_query search publications where date_inserted="2018-10-29"

Tot Publications:  0
Returned Publications: 0


<dimcli.Result object #4531148952: [('publications', 0), ('_stats', 1)]>

---

## DSL-107 

https://uberresearch.atlassian.net/browse/DSL-107

In [25]:
res = %dsl_query describe source grants 
res['fields']['funding_currency']

{'type': 'label',
 'description': 'Original funding currency',
 'long_description': None,
 'is_entity': False,
 'is_filter': True,
 'is_facet': True}

New fields does not appear...

In [28]:
res = %dsl_query search grants for "RDF" return funding_currency
res['funding_currency']

Tot Grants:  432
Returned Funding_currency: 12


[{'id': 'EUR', 'count': 101},
 {'id': 'USD', 'count': 91},
 {'id': 'JPY', 'count': 83},
 {'id': 'GBP', 'count': 32},
 {'id': 'CNY', 'count': 25},
 {'id': 'CAD', 'count': 20},
 {'id': 'PLN', 'count': 12},
 {'id': 'CHF', 'count': 6},
 {'id': 'HKD', 'count': 6},
 {'id': 'CZK', 'count': 5},
 {'id': 'SEK', 'count': 2},
 {'id': 'NOK', 'count': 1}]

In [34]:
%dsl_query search grants for "RDF" where funding_currency="JPY"


Tot Grants:  83
Returned Grants: 20


<dimcli.Result object #4529336048: [('grants', 20), ('_stats', 1)]>

In [35]:
%dsl_query search grants for "RDF" where funding_currency is empty return grants

Tot Grants:  48
Returned Grants: 20


<dimcli.Result object #4529570536: [('grants', 20), ('_stats', 1)]>

### VERDICT
Looks good

---

## DSL-97

https://uberresearch.atlassian.net/browse/DSL-97

In [38]:
%dsl_query search grants for "RDF" where funding_nzd is not empty return grants[funding_nzd] limit 1

Tot Grants:  354
Returned Grants: 1


<dimcli.Result object #4529585008: [('grants', 1), ('_stats', 3)]>

In [40]:
res = %dsl_query search grants where funding_nzd is not empty return grants limit 1

Tot Grants:  3701728
Returned Grants: 1


### VERDICT
Looks good

---

## DSL-118

https://uberresearch.atlassian.net/browse/DSL-118

new field altmetric_id

In [41]:
res = %dsl_query describe source publications 
res['fields']['altmetric_id']

{'type': 'integer',
 'description': 'AltMetric Publication ID',
 'long_description': None,
 'is_entity': False,
 'is_filter': True,
 'is_facet': False}

In [42]:
%dsl_query search publications for "ghana" where altmetric_id = "123"

Tot Publications:  0
Returned Publications: 0


<dimcli.Result object #4529766808: [('publications', 0), ('_stats', 1)]>

In [46]:
res = %dsl_query search publications where altmetric_id is not empty return publications[altmetric_id+title] limit 1 skip 500
res['publications']

Tot Publications:  10068170
Returned Publications: 1


[{'title': 'The in vivo specificity of synaptic Gβ and Gγ subunits to the α2a adrenergic receptor at CNS synapses',
  'altmetric_id': 55316946}]

In [47]:
res = %dsl_query search publications where altmetric_id="55316946" return publications[altmetric_id+title]
res['publications']

Tot Publications:  1
Returned Publications: 1


[{'title': 'The in vivo specificity of synaptic Gβ and Gγ subunits to the α2a adrenergic receptor at CNS synapses',
  'altmetric_id': 55316946}]

### VERDICT
Looks good

---

## DSL-108

https://uberresearch.atlassian.net/browse/DSL-108

research_org_state_names field is deprecated as a facet

In [48]:
res = %dsl_query describe source publications 
res['fields']['research_org_state_names']

{'type': 'string',
 'description': 'GeoNames state name',
 'long_description': None,
 'is_entity': False,
 'is_filter': True,
 'is_facet': False}

In [52]:
res = %dsl_query search publications where research_org_state_names~"China" return publications[research_org_state_names]
res['publications']

Tot Publications:  18
Returned Publications: 18


[{'research_org_state_names': ['Texas', 'Chinandega', 'San Salvador']},
 {'research_org_state_names': ['New York',
   'Massachusetts',
   'Chinandega',
   'Rivas']},
 {'research_org_state_names': ['Managua',
   'Chinandega',
   'Texas',
   'San Salvador']},
 {'research_org_state_names': ['Texas',
   'San Salvador',
   'Usulutan',
   'Chinandega']},
 {'research_org_state_names': ['Chinandega']},
 {'research_org_state_names': ['Chinandega']},
 {'research_org_state_names': ['Chinandega']},
 {'research_org_state_names': ['Chinandega']},
 {'research_org_state_names': ['Texas', 'Massachusetts', 'Chinandega']},
 {'research_org_state_names': ['Chinandega']},
 {'research_org_state_names': ['Illinois',
   'Florida',
   'Pennsylvania',
   'California',
   'Chinandega']},
 {'research_org_state_names': ['Lima', 'Chinandega', 'Managua']},
 {'research_org_state_names': ['Luxembourg',
   'Geneva',
   'Illinois',
   'Colorado',
   'Missouri',
   'Selangor',
   'Chinandega',
   'Pennsylvania',
   'Ontar

In [53]:
%dsl_query search publications where research_org_state_names~"China" return research_org_state_names


Semantic Error
Semantic errors found:
	Facet 'research_org_state_names' is not present in source 'publications'. Available facets are: FOR,FOR_first,HRCS_HC,HRCS_RAC,RCDC,funder_countries,funders,journal,mesh_terms,open_access_categories,publisher,referenced_pubs,research_org_cities,research_org_countries,research_org_state_codes,research_orgs,researchers,year


In [56]:
res = %dsl_query search publications where research_org_state_names~"China" return research_org_state_codes
res['research_org_state_codes']

Tot Publications:  18
Returned Research_org_state_codes: 13


[{'id': 'US-TX', 'count': 5, 'name': 'Texas'},
 {'id': 'US-MA', 'count': 3, 'name': 'Massachusetts'},
 {'id': 'US-CA', 'count': 2, 'name': 'California'},
 {'id': 'US-PA', 'count': 2, 'name': 'Pennsylvania'},
 {'id': 'CA-ON', 'count': 1, 'name': 'Ontario'},
 {'id': 'US-DC', 'count': 1, 'name': 'District of Columbia'},
 {'id': 'US-IA', 'count': 1, 'name': 'Iowa'},
 {'id': 'US-IN', 'count': 1, 'name': 'Indiana'},
 {'id': 'US-MN', 'count': 1, 'name': 'Minnesota'},
 {'id': 'US-MO', 'count': 1, 'name': 'Missouri'},
 {'id': 'US-NC', 'count': 1, 'name': 'North Carolina'},
 {'id': 'US-NY', 'count': 1, 'name': 'New York'},
 {'id': 'US-VA', 'count': 1, 'name': 'Virginia'}]

### VERDICT
* the functionality works correctly
* unsure how to correlated semantically research_org_state_codes with research_org_state_names... the examples above seem rather different. But this is more of a data question!

### PROBLEM
* if I filter for `research_org_state_codes.name` I don't get the same results as when I return `research_org_state_codes`

For example:

In [58]:
%dsl_query search publications where research_org_state_codes.name="Texas"

Tot Publications:  0
Returned Publications: 0


<dimcli.Result object #4530103296: [('publications', 0), ('_stats', 1)]>

---

## DSL-109

https://uberresearch.atlassian.net/browse/DSL-109

Disable facet recent_citations

In [57]:
res = %dsl_query describe source publications 
res['fields']['recent_citations']

{'type': 'integer',
 'description': 'Number of citations received in the last two years. Does not support emptiness filters',
 'long_description': None,
 'is_entity': False,
 'is_filter': True,
 'is_facet': False}

In [67]:
%dsl_query search publications where journal.title="Nature" return recent_citations

Semantic Error
Semantic errors found:
	Facet 'recent_citations' is not present in source 'publications'. Available facets are: FOR,FOR_first,HRCS_HC,HRCS_RAC,RCDC,funder_countries,funders,journal,mesh_terms,open_access_categories,publisher,referenced_pubs,research_org_cities,research_org_countries,research_org_state_codes,research_orgs,researchers,year


In [70]:
%dsl_query search publications where journal.title="Nature" return publications[doi+recent_citations] sort by recent_citations limit 3

Tot Publications:  664446
Returned Publications: 3


<dimcli.Result object #4530132360: [('publications', 3), ('_stats', 3)]>

### VERDICT
Looks good

---

## DSL-115

https://uberresearch.atlassian.net/browse/DSL-115

Disable faceting on `obsolete` field

In [94]:
res = %dsl_query describe source researchers 
res['fields']['obsolete']

{'type': 'integer',
 'description': 'Indicates researcher ID status. 0 means that the researcher ID is still active, 1 means that the researcher ID is no longer valid. See redirect field for further information on invalid researcher IDs. Also see function ``check_researcher_ids`` in :ref:`supported-functions`.',
 'long_description': None,
 'is_entity': False,
 'is_filter': True,
 'is_facet': False}

In [72]:
%dsl_query search researchers where first_grant_year="1999" return obsolete

Semantic Error
Semantic errors found:
	Facet 'obsolete' is not present in source 'researchers'. Available facets are: current_research_org,first_grant_year,first_publication_year,last_grant_year,last_publication_year,research_orgs,total_grants,total_publications


In [76]:
%dsl_query search researchers where first_grant_year="1999" and obsolete=0 return researchers[id+obsolete] limit 3 skip 4000 

Tot Researchers:  48177
Returned Researchers: 3


<dimcli.Result object #4530733240: [('researchers', 3), ('_stats', 3)]>

### VERDICT
* functionality is correct

### PROBLEMS
* update description text in JSON: remove \n and empty spaces
* in docs, add the following to the desc: "See also the check_researcher_ids function" (ideally with a link to https://docs.dimensions.ai/dsl/1.16.0-preview/functions.html#check-researcher-ids)

---

## DSL-112

https://uberresearch.atlassian.net/browse/DSL-112

Add terms to the grants source

In [77]:
res = %dsl_query describe source grants 
res['fields']['terms']

{'type': 'text',
 'description': 'Extracted terms',
 'long_description': None,
 'is_entity': False,
 'is_filter': False,
 'is_facet': False}

In [83]:
res = %dsl_query search grants in terms for "malaria" return grants[title+terms] limit 1 skip 500
res['grants']

Tot Grants:  8755
Returned Grants: 1


[{'title': 'SCH: INT: Collaborative Research: Smart Intervention Strategies for Hepatitis C Elimination',
  'terms': ['ongoing epidemic',
   'SCH',
   'survey',
   'long latency period',
   'collaborative research',
   'agents',
   'county health department',
   'fellows',
   'CDC',
   'true disease burden',
   'dynamic allocation',
   'decision makers',
   'research team',
   'practical decision support tool',
   'medicine',
   'simulation model',
   'control',
   'several challenges',
   'project',
   'students',
   'system',
   'approach',
   'application',
   'risk populations',
   'dynamic modeling',
   'effective intervention policies',
   'effective screening',
   'knowledge',
   'true prevalence',
   'HCV disease burden',
   'optimization',
   'learning',
   'national level',
   'disease',
   'health status',
   'infection',
   'few symptons',
   'epidemiology',
   'overarching aim',
   'ambitious targets',
   'adaptive intervention strategies',
   'tuberculosis',
   'several i

### VERDICT
* works fine 

### PROBLEMS
* remove sentence __See for "search term" regarding searching terms vs phrases"__ from [docs](https://docs.dimensions.ai/dsl/1.16.0-preview/data.html) description, as it is confusing
* we will deal with cleaning up the relationship between text search and terms search in the context of a different ticket, as soon as get agreement on the naming

---

## DSL-121

https://uberresearch.atlassian.net/browse/DSL-121

issue with long queries causing a recursion exception

In [95]:
%%dsl_query
search publications where (id="pub.1078412851" or pmid="21687907" or id="pub.1017692918" or doi="10.1017/epi.2012.11" or id="pub.1000947601" or doi="10.1353/lib.2014.0024" or id="pub.1005840286" or doi="10.1037/rel0000028" or id="pub.1072896184" or doi="10.5465/amle.2006.21253790" or id="pub.1008047826" or doi="10.1016/j.paid.2011.12.004" or id="pub.1059036018" or doi="10.1088/0143-0807/29/3/003" or id="pub.1073160696" or doi="10.5840/acpq20108415" or id="pub.1029258106" or doi="10.1080/09596410.2012.688259" or id="pub.1024363171" or doi="10.1016/j.jrp.2008.03.006" or id="pub.1064059641" or doi="10.1177/1548051809350892" or id="pub.1067630970" or doi="10.1521/jscp.23.5.620.50753" or id="pub.1025060184" or doi="10.1215/1089201x-2010-046" or id="pub.1032446800" or doi="10.1016/j.jrp.2009.01.003" or id="pub.1100929797" or doi="10.1177/009164711404200108" or id="pub.1100929801" or doi="10.1177/009164711404200112" or id="pub.1031824018" or doi="10.1007/s11089-011-0417-9" or id="pub.1040703453" or doi="10.1017/s0140525x10002888" or id="pub.1036369050" or doi="10.1080/15528030.2013.857380" or id="pub.1012305334" or doi="10.1163/1568533042650840" or id="pub.1031921162" or doi="10.2217/ahe.11.77" or id="pub.1036082057" or doi="10.1080/19349637.2015.957611" or id="pub.1076634202" or pmid="14761002" or id="pub.1040168037" or doi="10.1007/s11098-004-5751-7" or id="pub.1012672187" or doi="10.1016/j.jcps.2010.03.001" or id="pub.1010045665" or doi="10.1016/j.paid.2015.02.004" or id="pub.1008910323" or doi="10.1071/eg14006" or id="pub.1017891525" or doi="10.1016/j.paid.2015.02.017" or id="pub.1023582102" or doi="10.1525/jlin.2005.15.2.218" or id="pub.1029425398" or doi="10.1080/02604027.2013.803371" or id="pub.1070973004" or doi="10.3102/10769986030004353" or id="pub.1023147247" or doi="10.1080/14719037.2013.875392" or id="pub.1009259948" or doi="10.1016/j.radi.2011.05.002" or id="pub.1020700410" or doi="10.1046/j.1440-1614.1999.00583.x" or id="pub.1022459754" or doi="10.1177/0018726705059929" or id="pub.1023623880" or doi="10.1016/j.paid.2011.09.014" or id="pub.1021393674" or doi="10.1016/j.paid.2014.04.033" or id="pub.1007672608" or doi="10.1177/1357034x10373405" or id="pub.1067630268" or doi="10.1521/jscp.2006.25.5.i" or id="pub.1018288979" or doi="10.1016/j.jrp.2013.01.012" or id="pub.1077182929" or pmid="16450230" or id="pub.1100929625" or doi="10.1177/009164711003800202" or id="pub.1100929792" or doi="10.1177/009164711404200103" or id="pub.1100929799" or doi="10.1177/009164711404200110" or id="pub.1067687338" or doi="10.1525/lal.2013.25.1.20" or id="pub.1043577499" or doi="10.1007/s12126-009-9031-x" or id="pub.1074237633" or doi="10.1177/0731948711417556" or id="pub.1030141465" or doi="10.1111/j.1464-0597.2011.00445.x" or id="pub.1034708314" or doi="10.1111/j.1741-6612.2004.00022.x" or id="pub.1054829868" or doi="10.1017/s0036930612000117" or id="pub.1044208382" or doi="10.1016/j.paid.2012.07.005" or id="pub.1027428369" or doi="10.1057/dev.2009.52" or id="pub.1017172064" or doi="10.1007/s11116-013-9497-y" or id="pub.1063714008" or doi="10.1177/0090591705280525" or id="pub.1064025394" or doi="10.1177/14705931030032004" or id="pub.1000524486" or doi="10.1002/per.757" or id="pub.1003903591" or doi="10.1016/j.paid.2013.08.026" or id="pub.1022792380" or doi="10.1080/10464883.2013.817157" or id="pub.1045492518" or doi="10.1111/japp.12049" or id="pub.1034254211" or doi="10.1177/0966735009105872" or id="pub.1043893561" or doi="10.1007/s00300-008-0445-9" or id="pub.1033409695" or doi="10.1007/s11133-014-9289-5" or id="pub.1006858529" or doi="10.1016/j.paid.2013.09.018" or id="pub.1100929798" or doi="10.1177/009164711404200109" or id="pub.1038018965" or doi="10.1007/s11098-009-9352-3" or id="pub.1045167160" or doi="10.1007/s11406-014-9550-x" or id="pub.1072896499" or doi="10.5465/amle.2011.59513279" or id="pub.1064070004" or doi="10.1177/1742715012441178" or id="pub.1100929241" or doi="10.1177/009164710102900302" or id="pub.1017502370" or doi="10.1080/00207233.2010.517297" or id="pub.1013940401" or doi="10.1080/02582471003778391" or id="pub.1007109195" or doi="10.1598/jaal.54.5.6" or id="pub.1025219193" or doi="10.1002/per.830" or id="pub.1000878687" or doi="10.1016/j.paid.2013.10.031" or id="pub.1046125741" or doi="10.1111/1468-5906.00113" or id="pub.1072900464" or doi="10.5465/amj.2010.0441" or id="pub.1052882151" or doi="10.1007/s10591-013-9297-8" or id="pub.1067630169" or doi="10.1521/jscp.2000.19.1.70" or id="pub.1100929793" or doi="10.1177/009164711404200104" or id="pub.1063996642" or doi="10.1177/1362480610396646" or id="pub.1002425689" or doi="10.1080/10457097.2010.514558" or id="pub.1028439388" or doi="10.1111/ijst.12099" or id="pub.1043768808" or doi="10.1080/02615479.2013.805190" or id="pub.1017818842" or doi="10.1111/1467-6427.00068" or id="pub.1074760148" or pmid="11189464" or id="pub.1048256639" or doi="10.1016/j.jrp.2009.08.007" or id="pub.1064732923" or doi="10.1287/orsc.1120.0795" or id="pub.1015448068" or doi="10.1016/j.paid.2013.12.026" or id="pub.1036507015" or doi="10.1016/j.paid.2014.03.026" or id="pub.1073186851" or doi="10.5840/faithphil200118325" or id="pub.1100929796" or doi="10.1177/009164711404200107" or id="pub.1063576971" or doi="10.1177/0014524610363222" or id="pub.1007124640" or doi="10.1016/j.paid.2015.06.008" or id="pub.1029767528" or doi="10.1002/per.795" or id="pub.1002468198" or doi="10.1080/08959285.2014.913594" or id="pub.1006129682" or doi="10.1111/spc3.12069" or id="pub.1038539064" or doi="10.1111/j.0269-1213.2004.00071.x" or id="pub.1030378455" or doi="10.1007/s11406-015-9582-x" or id="pub.1041274188" or doi="10.1163/17455197-01202003" or id="pub.1008064266" or doi="10.1080/10705422.2010.490161" or id="pub.1036598408" or doi="10.1080/13674676.2015.1038229" or id="pub.1078553046" or pmid="22533053" or id="pub.1015265880" or doi="10.1016/j.paid.2011.06.011" or id="pub.1000063590" or doi="10.1080/13528165.2013.818322" or id="pub.1100929790" or doi="10.1177/009164711404200101" or id="pub.1023568914" or doi="10.1057/ap.2013.19" or id="pub.1027686749" or doi="10.1016/j.jrp.2015.07.002" or id="pub.1032278027" or doi="10.1177/1477878512437472" or id="pub.1031623673" or doi="10.1177/0022167803256979" or id="pub.1024790229" or doi="10.1111/j.1468-2400.2010.00524.x" or id="pub.1071024869" or doi="10.3138/utq.80.4.858" or id="pub.1052944431" or doi="10.1177/1948550613484179" or id="pub.1032457028" or doi="10.1007/s11406-006-9032-x" or id="pub.1024146244" or doi="10.1177/107769900608300311" or id="pub.1064141251" or doi="10.1190/1.1651456" or id="pub.1038887111" or doi="10.1525/aeq.2000.31.4.449" or id="pub.1025169219" or doi="10.1163/15700690-12341251" or id="pub.1100929795" or doi="10.1177/009164711404200106" or id="pub.1037783436" or doi="10.1016/j.paid.2011.01.011" or id="pub.1056347546" or doi="10.1027/1866-5888/a000108" or id="pub.1068186558" or doi="10.1654/4424.1" or id="pub.1100929791" or doi="10.1177/009164711404200102" or id="pub.1053174663" or doi="10.1111/1468-0114.00172" or id="pub.1100929732" or doi="10.1177/009164711204000303" or id="pub.1013880732" or doi="10.1111/j.1095-9270.2010.00295.x" or id="pub.1038295553" or doi="10.1007/s11007-009-9102-4" or id="pub.1049737583" or doi="10.1177/1948550614534700" or id="pub.1027375881" or doi="10.1007/s10943-005-9007-4" or id="pub.1030380605" or doi="10.1016/j.paid.2012.11.008" or id="pub.1051043573" or doi="10.1177/0001839213520131" or id="pub.1100929800" or doi="10.1177/009164711404200111" or id="pub.1036328411" or doi="10.1002/per.1860" or id="pub.1052303795" or doi="10.1080/01440357.2013.830833" or id="pub.1073168834" or doi="10.5840/augstudies201344228" or id="pub.1015202705" or doi="10.1016/j.paid.2014.02.043" or id="pub.1000632555" or doi="10.1016/j.leaqua.2014.11.005" or id="pub.1021272222" or doi="10.1080/00377310109517622" or id="pub.1047545963" or doi="10.1177/1948550610370717" or id="pub.1067630703" or doi="10.1521/jscp.2013.32.8.859" or id="pub.1014628109" or doi="10.1017/s1356186316000109" or id="pub.1045830767" or doi="10.1111/soru.12083" or id="pub.1041125836" or doi="10.1111/meta.12213" or id="pub.1013084383" or doi="10.1007/s11841-013-0381-1" or id="pub.1071629927" or doi="10.3917/litt.176.0047" or id="pub.1038515093" or doi="10.1080/15427609.2016.1166014" or id="pub.1029045952" or doi="10.1891/2156-5287.5.4.188" or id="pub.1049734074" or doi="10.1111/j.1365-2206.2009.00668.x" or id="pub.1028667769" or doi="10.1016/j.eurger.2012.06.007" or id="pub.1043365790" or doi="10.1016/j.paid.2015.08.030" or id="pub.1008864847" or doi="10.1016/j.paid.2016.03.043" or id="pub.1016219350" or doi="10.1007/978-3-662-47470-9_15" or id="pub.1019233969" or doi="10.1002/sdr.261" or id="pub.1032569091" or doi="10.1080/10720537.2012.679128" or id="pub.1073080761" or doi="10.5708/ejmh.11.2016.1-2.2" or id="pub.1036473350" or doi="10.1002/bdm.677" or id="pub.1016161006" or doi="10.1080/02703149.2011.580687" or id="pub.1011271771" or doi="10.1080/02615479.2015.1059805" or id="pub.1013294403" or doi="10.1007/s10670-005-3225-8" or id="pub.1009303453" or doi="10.1016/j.paid.2012.07.010" or id="pub.1049969342" or doi="10.1207/s15327043hup1802_4" or id="pub.1014673052" or doi="10.1080/08995605.2011.570589" or id="pub.1023251495" or doi="10.1016/j.paid.2014.05.007" or id="pub.1020634014" or doi="10.1163/18722636-12341284" or id="pub.1000205451" or doi="10.1017/eso.2015.64" or id="pub.1068330963" or doi="10.17323/1814-9545-2015-2-244-278" or id="pub.1084691508" or doi="10.1080/02615479.2017.1297393" or id="pub.1005889680" or doi="10.1002/per.733" or id="pub.1024153375" or doi="10.1007/978-90-481-2979-9_17" or id="pub.1040411204" or doi="10.1002/per.750" or id="pub.1016521547" or doi="10.1080/17439760.2015.1117130" or id="pub.1024990625" or doi="10.1007/s11245-015-9324-5" or id="pub.1009131023" or doi="10.1080/14780038.2016.1166410" or id="pub.1028428397" or doi="10.1017/s1740022807002033" or id="pub.1067428281" or doi="10.1504/ejim.2012.047038" or id="pub.1041839965" or doi="10.1002/per.1940" or id="pub.1040192262" or doi="10.1111/edth.12157" or id="pub.1025875044" or doi="10.1007/s12144-015-9310-5" or id="pub.1014245561" or doi="10.1016/j.jrp.2016.08.004" or id="pub.1072896003" or doi="10.5465/amle.2003.9324027" or id="pub.1017766835" or doi="10.1002/per.597" or id="pub.1063892836" or doi="10.1177/0957926508092244" or id="pub.1008343476" or doi="10.1016/j.paid.2012.06.010" or id="pub.1042356945" or doi="10.1111/1467-9256.12075" or id="pub.1046868396" or doi="10.1080/03057240.2016.1174675" or id="pub.1084529182" or doi="10.1016/j.jrp.2017.04.002" or id="pub.1012045652" or doi="10.1007/978-3-319-27929-9_1" or id="pub.1073250869" or doi="10.5840/socphiltoday201672731" or id="pub.1072894987" or doi="10.5465/ame.2004.15268751" or id="pub.1073160412" or doi="10.5840/acpq200579112" or id="pub.1011561620" or doi="10.1016/j.tate.2006.12.024" or id="pub.1100929853" or doi="10.1177/009164711504300403" or id="pub.1045193539" or doi="10.1080/17439760.2015.1037861" or id="pub.1010978278" or doi="10.1016/j.spacepol.2016.05.003" or id="pub.1058392338" or doi="10.1080/14623943.2016.1269003" or id="pub.1006175865" or doi="10.1080/10474412.2016.1167608" or id="pub.1054829225" or doi="10.1017/s0036930603001261" or id="pub.1022618163" or doi="10.1111/j.1467-9809.2010.01030.x" or id="pub.1032853834" or doi="10.1080/0046760x.2015.1136358" or id="pub.1016838189" or doi="10.1016/j.paid.2016.07.043" or id="pub.1045461276" or doi="10.1111/josi.12175" or id="pub.1009476753" or doi="10.1108/jgm-03-2015-0007" or id="pub.1074798326" or pmid="11299694" or id="pub.1064584762" or doi="10.1260/174795408785100635" or id="pub.1040414284" or doi="10.1111/j.1467-9973.2007.00501.x" or id="pub.1010094777" or doi="10.1016/j.paid.2007.04.008" or id="pub.1032663458" or doi="10.1111/j.1750-0206.2011.00283.x" or id="pub.1010885014" or doi="10.1353/sce.2016.0033" or id="pub.1014321092" or doi="10.1016/j.paid.2013.05.008" or id="pub.1056344172" or doi="10.1027/1015-5759.21.1.56" or id="pub.1001136461" or doi="10.1080/05568641.2011.560027" or id="pub.1043872366" or doi="10.1006/jrpe.2002.2342" or id="pub.1052030829" or doi="10.1016/j.jom.2009.01.003" or id="pub.1032962447" or doi="10.1108/01437731211203447" or id="pub.1018432473" or doi="10.1002/sres.2423" or id="pub.1063347740" or doi="10.11612/resphil.2016.93.3.10" or id="pub.1000159095" or doi="10.1177/0018726701543003" or id="pub.1093797356" or doi="10.1109/fie.2011.6142907" or id="pub.1044181675" or doi="10.1080/17531055.2014.916557" or id="pub.1020539693" or doi="10.1080/0267303042000152168" or id="pub.1048596729" or doi="10.1080/00048400601154376" or id="pub.1043303377" or doi="10.1080/09500690902874894" or id="pub.1047463603" or doi="10.1080/1070289x.2014.873043" or id="pub.1019314286" or doi="10.1080/17439760.2016.1167938" or id="pub.1036637416" or doi="10.1080/17439760.2016.1167944" or id="pub.1073099235" or doi="10.5752/p.2175-5841.2016v14n43p1064" or id="pub.1085296070" or doi="10.1080/09552367.2017.1319084" or id="pub.1036298750" or doi="10.1016/j.paid.2011.05.025" or id="pub.1024837555" or doi="10.1179/1474893213z.00000000081" or id="pub.1001718097" or doi="10.1515/zaa-2014-0006" or id="pub.1068330522" or doi="10.17323/1726-3247-2015-5-118-139" or id="pub.1084100743" or doi="10.1016/j.paid.2017.02.056" or id="pub.1025007146" or doi="10.1080/13507480903368087" or id="pub.1044086127" or doi="10.1080/02582470903500442" or id="pub.1045791369" or doi="10.3406/rbph.2009.7704" or id="pub.1059651490" or doi="10.1093/hwj/dbs055" or id="pub.1031710958" or doi="10.1080/15022250.2015.1006390" or id="pub.1038955697" or doi="10.1016/j.hrmr.2015.01.006" or id="pub.1000151768" or doi="10.1080/00076791.2016.1172569" or id="pub.1021123890" or doi="10.1016/j.paid.2016.04.096" or id="pub.1035317010" or doi="10.1017/s0140525x00233438" or id="pub.1017708008" or doi="10.1177/0002716203254762" or id="pub.1043551523" or doi="10.1111/j.1467-8373.2008.00379.x" or id="pub.1028146237" or doi="10.1016/j.paid.2009.05.029" or id="pub.1071184599" or doi="10.3366/e026483340900056x" or id="pub.1030133589" or doi="10.1007/s11191-011-9346-3" or id="pub.1008348129" or doi="10.1016/j.paid.2013.08.037" or id="pub.1002104668" or doi="10.1080/14742837.2013.860025" or id="pub.1034889795" or doi="10.1037/a0034163" or id="pub.1052215622" or doi="10.1080/03057240.2016.1168736" or id="pub.1073160990" or doi="10.5840/acpq201531749" or id="pub.1089280541" or doi="10.1057/9781137500328.0010" or id="pub.1085401832" or doi="10.1017/s1744552317000118" or id="pub.1100276579" or doi="10.1080/00222216.2001.11949938" or id="pub.1017321096" or doi="10.1016/j.futures.2009.11.028" or id="pub.1028338184" or doi="10.1007/s10862-010-9210-9") return publications[id+doi+pmid] limit 1000

1 QueryError found

Query too long. See https://docs.dimensions.ai/dsl/faq.html for more info on troubleshooting common errors


### VERDICT
* works fine 

---

## DSL-126

https://uberresearch.atlassian.net/browse/DSL-126

grants query error due to long subquery

In [98]:
%dsl_query search grants in full_data for "\"lung cancer\"" where researchers.last_name="Wang" return grants [all] limit 250 skip 0

Tot Grants:  11994
Returned Grants: 20


<dimcli.Result object #4530701872: [('grants', 20), ('_stats', 1)]>